```python hl_lines 9
print('hello')
```

In this post, we'll walk through the math inside the original paper that discovered diffusion models: _Deep Unsupervised Learning using Nonequilibrium Thermodynamics_ [@sohl-dickstein_deep_2015].

Diffusion models[^name] (DMs) are a new class of _deep generative models_. At a high level, the goal of generative modeling is to learn how to create new data points that are similar to a given dataset. For example, given the MNIST dataset---a collection of small black-and-white pictures of digits---a generative model would be able to create entirely pictures of digits that look similar to those in the original dataset. _Deep_ generative models are those that use deep learning methods. Other classes of deep generative models include variational auto-encoders (VAEs) and generative adversarial networks (GANs).

[^name]: While the authors used the name "diffusion probabilistic models", they're more commonly known as "diffusion models" today.

Before we begin it's important to note that you don't need to understand the math that inspired diffusion models to use them effectively, or even to improve them with novel research. In fact, many improvements following their introduction did so by deviating from the original mathematical formulation! I wrote this because I think that the math is beautiful. I wanted to understand that beauty myself (as a non-mathematician) and to make it accessible to others.

# Forward trajectory

The method is described in the second section. It starts by describing a "forward trajectory" in a rather dense paragraph. It's common to see mathematical writing as dense as this (and often more). This brevity allows mathematicians to develop new ideas. By assuming that the reader is familiar with all of the prerequisites, the authors are able to focus on only the new parts. The trade-off is that new readers have to do a lot more work upfront.

There are four objects introduced here, highlighted in four different colors. The two equations describe the relationships between these objects:

::: {.pt-3 .pb-1 .px-3 .my-3 .border .rounded .shadow-sm}

We label the [data distribution $q(\mathbf{x}^{(0)})$]{.hl-indigo}. The data distribution is gradually converted into a [well behaved (analytically tractable) distribution $\pi(\mathbf y)$]{.hl-green} by repeated application of a [Markov diffusion kernel $T_\pi(\mathbf y|\mathbf y'; \beta)$]{.hl-yellow} for $\pi(\mathbf y)$, where [$\beta$ is the diffusion rate]{.hl-blue},

\begin{align}
\pi(\mathbf y) &= \int d\mathbf y' T_\pi (\mathbf y|\mathbf y';\beta) \pi(\mathbf y') \tag{1} \\
q(\mathbf x^{(t)}|\mathbf x^{(t-1)}) &= T_\pi(\mathbf x^{(t)} | \mathbf x^{(t-1)}; \beta_t) \tag{2}
\end{align}

:::

## Random variables vs probability density functions

The first concept required to understand this is that of representing random variables (vectors in this case) in terms of their _probability density functions_. Mathematicians do this to enable the analysis of random variables. Although it isn't explicitly mentioned here, $\pi$, $q$, and $T_\pi$ are all probability density functions. The pdf $\pi(\mathbf y)$ is a function that accepts a vector as input and returns the probability that the provided value of $\mathbf y$ might occur.

Pdfs have been studied for a very long time, so they have many properties which are now well understood by mathematicians.

The authors of this paper use the notation that the input symbol of the pdf is the name of the random vector described by said pdf. For example, $\mathbf y$ is a _random vector_ whose behavior is completely described by the probability density function $\pi$.

Coders often find it easier to think in terms of the random variables and vectors, however, since they're closer to what gets written into our programs in the end. Let's shift focus to the random vectors introduced here.

## Data distribution $q(\mathbf x^{(0)})$

There are a few more subtleties in the notation used for these random vectors. The first random vector we were introduced to is $\mathbf x^{(0)}$. $x$ is often used for the object that's ultimately being modeled. In this case, it's a random vector representing the _data distribution_. For example, if we were using the MNIST dataset, then $\mathbf x^{(0)}$ is a randomly sampled image that looks like an MNIST digit. It may not be an actual image from the dataset -- this is the goal of the abstraction "data distribution", it assumes some underlying data generating system, of which the actual MNIST dataset is only one sample.

The superscript $(0)$ here does not mean "to the power of". This is suggested by the brackets surrounding the zero, a notation commonly used in some domains including physics. The use of the superscript already suggests that we might see the sequence continue, for example, $\mathbf x^{(1)}$, $\mathbf x^{(2)}$, and so on.

## Analytically tractable distribution $\pi(\mathbf y)$

The use of $\mathbf y$ in $\pi(\mathbf y)$ suggests that the authors are trying to highlight a more general property about the functions $\pi$ and $T_\pi$ that's totally unrelated to the data $\mathbf x$.

What does "analytically tractable" distribution mean? I'm not 100% sure but I think it means a relatively straightforward formula that we can type out in Python when defining a simple function. In other words, the authors claim that we can gradually convert the data distribution (some hypothetical Python function that returns novel images that look like MNIST digits) into a relatively straightforward mathematical formula that can be programmed in Python!

If we peak ahead to Table App. 1[^peaking], we can see some examples of these objects. However, instead of $\pi(\mathbf y)$ we now see $\pi(\mathbf x^{(T)})$, which may be set to $\mathcal N (\mathbf x^{(T)}; \mathbf 0, \mathbf I)$ (Gaussian) or $\mathcal B(\mathbf x^{(T)}; 0.5)$ (Binomial).

This is a new notation, let's describe it. $\mathcal N$ and $\mathcal B$ refer to pdfs of two well-known distributions: Gaussian and Binomial. The semicolon separates function inputs on the left from parameters on the right. It's not immediately obvious what the difference is between an input and parameter of a function, they only really make sense for classes of functions aka models. For example, the equation of a Gaussian pdf can be written down as a function of the inputs as well as two parameters. Recall from earlier that this is why we might call it an analytically tractable distribution.

[^peaking]: It's often a good idea to peak ahead while reading a scientific paper. In fact, I often find it most useful to jump back and forth many times and very rarely read papers linearly. I was introduced to this and other ideas in the wonderful [_How to Read a Paper_](https://web.stanford.edu/class/ee384m/Handouts/HowtoReadPaper.pdf) by Srinivasan Keshav.

## Equation 1

$$
\pi(\mathbf y) = \int d\mathbf y' T_\pi (\mathbf y|\mathbf y';\beta) \pi(\mathbf y') \tag{1}
$$